In [1]:
"""
Esta celda:
- Configura rutas del proyecto.
- Apunta MLflow a una base nueva (mlflow_new.db) para evitar el error de migración.
- No entrena nada, solo prepara el registro.
"""

import json
from pathlib import Path
from datetime import datetime

import torch
import mlflow
from mlflow.tracking import MlflowClient

PROJECT_ROOT = Path(r"C:\Users\Johnny\Desktop\IA-final").resolve()
MODELS_DIR = (PROJECT_ROOT / "models" / "local_checkpoints").resolve()
PROCESSED_DIR = (PROJECT_ROOT / "data" / "processed").resolve()

PROJECT_CONFIG_PATH = (PROCESSED_DIR / "project_config.json").resolve()
LABELMAP_PATH = (PROCESSED_DIR / "labelmap.json").resolve()

# checkpoint entrenado (BEST)
CKPT_PATH = MODELS_DIR / "best_frcnn_cpu_base_train_20260201_083448.pt"
if not CKPT_PATH.exists():
    raise FileNotFoundError(f"No existe CKPT_PATH: {CKPT_PATH}")

# MLflow: DB nueva
MLFLOW_DB = (PROJECT_ROOT / "mlflow_new.db").resolve()
mlflow.set_tracking_uri(f"sqlite:///{MLFLOW_DB.as_posix()}")

EXPERIMENT_NAME = "object_detection_coco_cpu"
mlflow.set_experiment(EXPERIMENT_NAME)

client = MlflowClient()

print("PROJECT_ROOT:", PROJECT_ROOT)
print("CKPT_PATH:", CKPT_PATH)
print("MLFLOW_DB:", MLFLOW_DB)
print("EXPERIMENT_NAME:", EXPERIMENT_NAME)


2026/02/02 09:50:56 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/02 09:50:56 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

PROJECT_ROOT: C:\Users\Johnny\Desktop\IA-final
CKPT_PATH: C:\Users\Johnny\Desktop\IA-final\models\local_checkpoints\best_frcnn_cpu_base_train_20260201_083448.pt
MLFLOW_DB: C:\Users\Johnny\Desktop\IA-final\mlflow_new.db
EXPERIMENT_NAME: object_detection_coco_cpu


In [2]:
"""
Esta celda:
- Carga el checkpoint y extrae metadata (config, classes, best_val_loss si existe).
- Crea un run en MLflow y sube el checkpoint como artefacto.
- Crea/actualiza el Registered Model y promueve esta versión a Production.
- No requiere entrenamiento.
"""

REGISTERED_MODEL_NAME = "frcnn_coco_cpu_person_car_airplane"

# crear registered model si no existe
existing = [m.name for m in client.search_registered_models()]
if REGISTERED_MODEL_NAME not in existing:
    client.create_registered_model(REGISTERED_MODEL_NAME)

ckpt = torch.load(CKPT_PATH, map_location="cpu")

config = ckpt.get("config", {})
target_classes = ckpt.get("target_classes", [])
epoch = ckpt.get("epoch", None)
best_val_loss = ckpt.get("best_val_loss", None)

run_name = f"recover_base_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

with mlflow.start_run(run_name=run_name) as run:
    run_id = run.info.run_id

    # tags
    mlflow.set_tag("stage", "recovered")
    mlflow.set_tag("status", "RESTORED_FROM_CHECKPOINT")
    mlflow.set_tag("checkpoint_file", CKPT_PATH.name)
    if target_classes:
        mlflow.set_tag("classes", ",".join(target_classes))
    mlflow.set_tag("model_arch", "fasterrcnn_resnet50_fpn")

    # params
    if isinstance(config, dict) and config:
        mlflow.log_params(config)

    # metrics si existen
    if epoch is not None:
        mlflow.log_metric("epoch", float(epoch))
    if best_val_loss is not None:
        mlflow.log_metric("best_val_loss", float(best_val_loss))

    # artefactos relevantes
    mlflow.log_artifact(str(CKPT_PATH), artifact_path="model_ckpt")
    if PROJECT_CONFIG_PATH.exists():
        mlflow.log_artifact(str(PROJECT_CONFIG_PATH), artifact_path="artifacts")
    if LABELMAP_PATH.exists():
        mlflow.log_artifact(str(LABELMAP_PATH), artifact_path="artifacts")

    # crear versión del modelo en registry apuntando al artefacto del run
    model_uri = f"runs:/{run_id}/model_ckpt/{CKPT_PATH.name}"
    mv = client.create_model_version(
        name=REGISTERED_MODEL_NAME,
        source=model_uri,
        run_id=run_id
    )

    # tags de la versión
    client.set_model_version_tag(REGISTERED_MODEL_NAME, mv.version, "recovered_from_ckpt", CKPT_PATH.name)
    if target_classes:
        client.set_model_version_tag(REGISTERED_MODEL_NAME, mv.version, "classes", ",".join(target_classes))

    # promover a Production y archivar anteriores
    client.transition_model_version_stage(
        name=REGISTERED_MODEL_NAME,
        version=mv.version,
        stage="Production",
        archive_existing_versions=True
    )

print("OK: modelo recuperado y registrado sin reentrenar.")
print("Registered model:", REGISTERED_MODEL_NAME)
print("Version promoted to Production:", mv.version)
print("Tracking DB:", MLFLOW_DB)


OK: modelo recuperado y registrado sin reentrenar.
Registered model: frcnn_coco_cpu_person_car_airplane
Version promoted to Production: 1
Tracking DB: C:\Users\Johnny\Desktop\IA-final\mlflow_new.db


C:\Users\Johnny\AppData\Local\Temp\ipykernel_17556\1130130834.py:67: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
